In [4]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import spacy
import random
import numpy as np
from spacy import tokenizer
import re

In [66]:
!python -m spacy download en_core_web_md

     |████████████████████████████████| 120.9MB 1.3MB/s eta 0:00:01
  Created wheel for en-core-web-md: filename=en_core_web_md-2.0.0-cp36-none-any.whl size=122523224 sha256=ed1dc69050da26112d2caa2f7f9bc5b91feb4f0a129b41b90d89f66bb46f24cb
  Stored in directory: /private/var/folders/gs/0q9bycyx5kq5qtmt4q6qq9mh0000gn/T/pip-ephem-wheel-cache-5ok_6onm/wheels/db/5d/d0/ccdad6b01f9695b4a33793158530b3228223ee31463441663d
Successfully built en-core-web-md

    Linking successful
    /Users/mkorakakis/miniconda3/envs/nlp/lib/python3.6/site-packages/en_core_web_md
    -->
    /Users/mkorakakis/miniconda3/envs/nlp/lib/python3.6/site-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [70]:
nlp = spacy.load("en_core_web_sm")

In [71]:
vqa = json.load(open("OpenEnded_mscoco_val2014_questions.json"))['questions']
df = json_normalize(vqa)

In [72]:
df.columns

Index(['question', 'image_id', 'question_id'], dtype='object')

In [73]:
df

,question,image_id,question_id
0,What is the table made of?,350623,3506232
1,Is the food napping on the table?,350623,3506230
2,What has been upcycled to make lights?,350623,3506231
3,Is this an Spanish town?,8647,86472
4,Are there shadows on the sidewalk?,8647,86470
...,...,...,...
121507,What is holding up the phone?,174503,1745030
121508,What kind of glasses is she wearing?,174503,1745031
121509,What time does the clock say on the stove?,552610,5526100
121510,Does the kitchen appear to be clean?,552610,5526101


In [74]:
questions = df['question']
print(questions[1])
print(questions)

Is the food napping on the table?
0                         What is the table made of?
1                  Is the food napping on the table?
2             What has been upcycled to make lights?
3                           Is this an Spanish town?
4                 Are there shadows on the sidewalk?
                             ...                    
121507                 What is holding up the phone?
121508          What kind of glasses is she wearing?
121509    What time does the clock say on the stove?
121510          Does the kitchen appear to be clean?
121511           Is there water in the water bottle?
Name: question, Length: 121512, dtype: object


In [75]:
df.dtypes

question       object
image_id        int64
question_id     int64
dtype: object

In [78]:
def add_to_dict(image_id, question, question_id):
    perturbed_dict = {}

    perturbed_dict["image_id"] = int(image_id)
    question = ' '.join(question[:-1])
    question += '?'
    perturbed_dict["question"] = question
    perturbed_dict["question_id"] = int(question_id)
    
    return perturbed_dict

In [79]:
json_dict = {}
perturbed_questions = []

for i in range(len(questions)):
    doc = nlp(questions[i])
    drop_wh = []
    for word, token in enumerate(doc):
        if token.tag_ not in ['WDT', 'WP', 'WP$', 'WRB']:
            drop_wh.append(token.text)
            
    row_data = df.loc[i, df.columns != 'question']
    perturbed_questions.append(add_to_dict(row_data['image_id'], drop_wh, row_data['question_id']))    

json_dict["questions"] = perturbed_questions
with open("vqa_drop_wh.json", "w") as output_file:
    json.dump(json_dict, output_file)

In [80]:
json_dict = {}
perturbed_questions = []


for i in range(len(questions)):
    doc = nlp(questions[i])
    
    drop_nouns = []
    for word, token in enumerate(doc):
        if token.tag_ not in ['NN', 'NNP', 'NNPS', 'NNS']:
            drop_nouns.append(token.text)
            
    row_data = df.loc[i, df.columns != 'question']
    perturbed_questions.append(add_to_dict(row_data['image_id'], drop_nouns, row_data['question_id']))

json_dict["questions"] = perturbed_questions
with open("vqa_drop_nouns.json", "w") as outfile:
    json.dump(json_dict, outfile)

KeyboardInterrupt: 

In [ ]:
json_dict = {}
perturbed_questions = []


for i in range(len(questions)):
    doc = nlp(questions[i])
    

    drop_verbs = []
    for word, token in enumerate(doc):
        if token.pos_ is not 'VERB':
            drop_verbs.append(token.text)
   
    row_data = df.loc[i, df.columns != 'question']
    perturbed_questions.append(add_to_dict(row_data['image_id'], drop_verbs, row_data['question_id']))
    
json_dict["questions"] = perturbed_questions
with open("vqa_drop_verbs.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [ ]:
json_dict = {}
perturbed_questions = []


for i in range(len(questions)):

    words_to_keep = int(len(questions[i].split())*(1-0.4))
    question = ' '.join(words[:words_to_keep])
    question += '?'
    
    
    row_data = df.loc[i, df.columns != 'question']    
    perturbed_dict = {}
    perturbed_dict["image_id"] = int(row_data['image_id'])
    perturbed_dict["question"] = question
    perturbed_dict["question_id"] = int(row_data['question_id'])
    
    perturbed_questions.append(perturbed_dict)
    
json_dict["questions"] = perturbed_questions
with open("vqa_remove_words.json", "w") as outfile:
    json.dump(json_dict, outfile)

In [ ]:
json_dict = {}
perturbed_questions = []


for i in range(len(questions)):

    question = questions[i].replace("?","").split()
    shuffled = random.sample(question, len(question))
    question = ' '.join(shuffled)
    question += '?'
    
    
    row_data = df.loc[i, df.columns != 'question']    
    perturbed_dict = {}
    perturbed_dict["image_id"] = int(row_data['image_id'])
    perturbed_dict["question"] = question
    perturbed_dict["question_id"] = int(row_data['question_id'])
    
    perturbed_questions.append(perturbed_dict)
    
json_dict["questions"] = perturbed_questions
with open("vqa_shuffle_words.json", "w") as outfile:
    json.dump(json_dict, outfile)

# Reading Comprehension

In [64]:
qs = json.load(open("dev-v2.0.json"))['data']
df = json_normalize(qs)

In [65]:
df.columns

Index(['title', 'paragraphs'], dtype='object')

In [66]:
df

,title,paragraphs
0,Normans,[{'qas': [{'question': 'In what country is Nor...
1,Computational_complexity_theory,[{'qas': [{'question': 'What branch of theoret...
2,Southern_California,[{'qas': [{'question': 'What is Southern Calif...
3,Sky_(United_Kingdom),[{'qas': [{'question': 'What company was forme...
4,Victoria_(Australia),[{'qas': [{'question': 'What kind of economy d...
5,Huguenot,"[{'qas': [{'question': ""Where was France's Hug..."
6,Steam_engine,[{'qas': [{'question': 'Along with geothermal ...
7,Oxygen,[{'qas': [{'question': 'The atomic number of t...
8,1973_oil_crisis,[{'qas': [{'question': 'When did the 1973 oil ...
9,European_Union_law,[{'qas': [{'question': 'What is European Union...


In [ ]:
dfToList = df['question'].tolist()
dfToList

In [68]:
def load_squad(data_path):

    with open(data_path) as f:
        data = json.load(f)['data']
    articles = []
    for article in data:
        print(article)
        article_dict= {}
        article_dict['title'] = article["title"]
        
        paragraph_dict = []
        for paragraph in article['paragraphs']:
            
            
            questions = []
            for qa in paragraph['qas']:
                
                drop_nouns = []
                question = nlp(qa['question'])
                for word, token in enumerate(question):
                    if token.tag_ not in ['NN', 'NNP', 'NNPS', 'NNS']:
                        drop_nouns.append(token.text)
                words_dict = {}
                words_dict["id"] = qa['id']
                words_dict["question"] = ' '.join(drop_nouns)
                words_dict['answers'] = []
                words_dict['is_impossible'] = qa['is_impossible']
                if "answers" in qa:
                    words_dict['answers'] = qa['answers']                
                questions.append(words_dict)
            question_dict = {}
            question_dict["qas"] = questions
            question_dict["context"] = paragraph['context']
            paragraph_dict.append(question_dict)
        article_dict['paragraphs'] = paragraph_dict
        articles.append(article_dict)


    json_dict["squad_drop_nouns"] = articles
    with open("abcfffff.json", "w") as output_file:
        json.dump(json_dict, output_file)




In [ ]:
def load_perturbed_squad(data_path):

    with open(data_path) as f:
        data = json.load(f)['data']
    articles = []
    for article in data:
        print(article)
        article_dict= {}
        article_dict['title'] = article["title"]
        
        paragraph_dict = []
        for paragraph in article['paragraphs']:
            
            
            questions = []
            for qa in paragraph['qas']:
                
                drop_nouns = []
                question = nlp(qa['question'])
                for word, token in enumerate(question):
                    if token.tag_ not in ['WDT', 'WP', 'WP$', 'WRB']:
                        drop_nouns.append(token.text)
                words_dict = {}
                words_dict["id"] = qa['id']
                words_dict["question"] = ' '.join(drop_nouns)
                words_dict['answers'] = []
                words_dict['is_impossible'] = qa['is_impossible']
                if "answers" in qa:
                    words_dict['answers'] = qa['answers']                
                questions.append(words_dict)
            question_dict = {}
            question_dict["qas"] = questions
            question_dict["context"] = paragraph['context']
            paragraph_dict.append(question_dict)
        article_dict['paragraphs'] = paragraph_dict
        articles.append(article_dict)


    json_dict["squad_drop_wh_words"] = articles
    with open("abcfffff.json", "w") as output_file:
        json.dump(json_dict, output_file)





In [ ]:
def load_perturbed_squad(data_path):

    with open(data_path) as f:
        data = json.load(f)['data']
    articles = []
    for article in data:
        print(article)
        article_dict= {}
        article_dict['title'] = article["title"]
        
        paragraph_dict = []
        for paragraph in article['paragraphs']:
            
            
            questions = []
            for qa in paragraph['qas']:
                
                drop_nouns = []
                question = nlp(qa['question'])
                for word, token in enumerate(question):
                    if token.pos_ is not 'VERB':
                        drop_nouns.append(token.text)
                words_dict = {}
                words_dict["id"] = qa['id']
                words_dict["question"] = ' '.join(drop_nouns)
                words_dict['answers'] = []
                words_dict['is_impossible'] = qa['is_impossible']
                if "answers" in qa:
                    words_dict['answers'] = qa['answers']                
                questions.append(words_dict)
            question_dict = {}
            question_dict["qas"] = questions
            question_dict["context"] = paragraph['context']
            paragraph_dict.append(question_dict)
        article_dict['paragraphs'] = paragraph_dict
        articles.append(article_dict)


    json_dict["squad_drop_verbs"] = articles
    with open("abcfffff.json", "w") as output_file:
        json.dump(json_dict, output_file)






In [ ]:
def load_perturbed_squad(data_path):

    with open(data_path) as f:
        data = json.load(f)['data']
    articles = []
    for article in data:
        print(article)
        article_dict= {}
        article_dict['title'] = article["title"]
        
        paragraph_dict = []
        for paragraph in article['paragraphs']:
            
            
            questions = []
            for qa in paragraph['qas']:
                
                words_to_keep = int(len(qa['question'].split())*(1-0.4))
                question = ' '.join(words[:words_to_keep])
                question += '?'
                
                words_dict = {}
                words_dict["id"] = qa['id']
                words_dict["question"] = ' '.join(question)
                words_dict['answers'] = []
                words_dict['is_impossible'] = qa['is_impossible']
                if "answers" in qa:
                    words_dict['answers'] = qa['answers']                
                questions.append(words_dict)
            question_dict = {}
            question_dict["qas"] = questions
            question_dict["context"] = paragraph['context']
            paragraph_dict.append(question_dict)
        article_dict['paragraphs'] = paragraph_dict
        articles.append(article_dict)


    json_dict["squad_drop_words"] = articles
    with open("abcfffff.json", "w") as output_file:
        json.dump(json_dict, output_file)






In [ ]:
def load_perturbed_squad(data_path):

    with open(data_path) as f:
        data = json.load(f)['data']
    articles = []
    for article in data:
        print(article)
        article_dict= {}
        article_dict['title'] = article["title"]
        
        paragraph_dict = []
        for paragraph in article['paragraphs']:
            
            
            questions = []
            for qa in paragraph['qas']:
                
                question = qa['question'].replace("?","").split()
                shuffled = random.sample(question, len(question))
                question = ' '.join(shuffled)
                question += '?'
    
                words_dict = {}
                words_dict["id"] = qa['id']
                words_dict["question"] = ' '.join(drop_nouns)
                words_dict['answers'] = []
                words_dict['is_impossible'] = qa['is_impossible']
                if "answers" in qa:
                    words_dict['answers'] = qa['answers']                
                questions.append(words_dict)
            question_dict = {}
            question_dict["qas"] = questions
            question_dict["context"] = paragraph['context']
            paragraph_dict.append(question_dict)
        article_dict['paragraphs'] = paragraph_dict
        articles.append(article_dict)


    json_dict["questions"] = articles
    with open("abcfffff.json", "w") as output_file:
        json.dump(json_dict, output_file)






## TQA

In [129]:
f = open('plug.examples', 'a')
f.write("stuff")
f.close()

In [ ]:
def add_to_dict(image_id, question, question_id):
    perturbed_dict = {}

    perturbed_dict["image_id"] = int(image_id)
    question = ' '.join(question[:-1])
    question += '?'
    perturbed_dict["question"] = question
    perturbed_dict["question_id"] = int(question_id)
    
    return perturbed_dict

In [239]:
perturbed_inputs = []
with open('random-split-1-dev.examples') as f:
    lines = f.readlines()
    for line in lines:

        doc = nlp(re.findall(r'"(.*?)"', line)[0])
        print(doc)
        drop_wh = []
        for word, token in enumerate(doc):
            if token.tag_ not in ['WDT', 'WP', 'WP$', 'WRB']:
                drop_wh.append(token.text)
        
        question = ' '.join(drop_wh[:-1])
        question += '?'
         
        
        replaced = re.sub(r'utterance "(.*?)"', 'utterance "%s"' % question, line)
        perturbed_inputs.append(replaced)
        
        break

with open("drop_wh.examples", "a") as f:
    for line in perturbed_inputs:
        f.write(line)
        

which team won previous to crettyard?
<class 'str'>
(example (id nt-2) (utterance "team won previous to crettyard?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))

(example (id nt-2) (utterance "which team won previous to crettyard?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))

(example (id nt-2) (utterance "team won previous to crettyard?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))



In [241]:
perturbed_inputs = []
with open('random-split-1-dev.examples') as f:
    lines = f.readlines()
    for line in lines:

        doc = nlp(re.findall(r'"(.*?)"', line)[0])
        print(doc)
        drop_verbs = []
        for word, token in enumerate(doc):
            if token.pos_ is not 'VERB':
                drop_verbs.append(token.text)
        
        question = ' '.join(drop_verbs[:-1])
        question += '?'
         
        
        replaced = re.sub(r'utterance "(.*?)"', 'utterance "%s"' % question, line)
        perturbed_inputs.append(replaced)
        print(replaced)
        break

with open("drop_verbs.examples", "a") as f:
    for line in perturbed_inputs:
        f.write(line)
        

which team won previous to crettyard?
(example (id nt-2) (utterance "which team previous to crettyard?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))

(example (id nt-2) (utterance "which team previous to crettyard?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))



In [243]:
perturbed_inputs = []
with open('random-split-1-dev.examples') as f:
    lines = f.readlines()
    for line in lines:

        doc = nlp(re.findall(r'"(.*?)"', line)[0])
        print(doc)
        drop_nouns = []
        for word, token in enumerate(doc):
            if token.tag_ not in ['NN', 'NNP', 'NNPS', 'NNS']:
                drop_nouns.append(token.text)
        
        question = ' '.join(drop_nouns[:-1])
        question += '?'
         
        
        replaced = re.sub(r'utterance "(.*?)"', 'utterance "%s"' % question, line)
        perturbed_inputs.append(replaced)
        print(replaced)
        break

with open("drop_nouns.examples", "a") as f:
    for line in perturbed_inputs:
        f.write(line)
        

which team won previous to crettyard?
(example (id nt-2) (utterance "which won previous to?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))

(example (id nt-2) (utterance "which won previous to?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))



In [247]:
perturbed_inputs = []
with open('random-split-1-dev.examples') as f:
    lines = f.readlines()
    for line in lines:

        doc = nlp(re.findall(r'"(.*?)"', line)[0])
        words = doc.text.split()
        words_to_keep = int(len(words)*(1-0.4))
        question = ' '.join(words[:words_to_keep])
        question += '?'   
        
            
        replaced = re.sub(r'utterance "(.*?)"', 'utterance "%s"' % question, line)
        perturbed_inputs.append(replaced)
        
        break

with open("drop_words.examples", "a") as f:
    for line in perturbed_inputs:
        print(line)
        f.write(line)
        

which team won previous to crettyard?
(example (id nt-2) (utterance "which team won?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))



In [249]:
perturbed_inputs = []
with open('random-split-1-dev.examples') as f:
    lines = f.readlines()
    for line in lines:

        doc = nlp(re.findall(r'"(.*?)"', line)[0])

        question = doc.text
        question = question.replace("?","").split()
        shuffled = random.sample(question, len(question))
        question = ' '.join(shuffled)
        question += '?'
        
        replaced = re.sub(r'utterance "(.*?)"', 'utterance "%s"' % question, line)
        perturbed_inputs.append(replaced)
        
        break

with open("shuffle_words.examples", "a") as f:
    for line in perturbed_inputs:
        print(line)
        f.write(line)
        

(example (id nt-2) (utterance "previous team crettyard won which to?") (context (graph tables.TableKnowledgeGraph csv/204-csv/772.csv)) (targetValue (list (description "Wolfe Tones"))))

